In [15]:
# Cryptocurrency Analysis Script v002

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly.io import write_image

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2014, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/indicators/investor_capitalization',
    'https://api.glassnode.com/v1/metrics/supply/current',
    'https://api.glassnode.com/v1/metrics/indicators/liveliness',
    'https://api.glassnode.com/v1/metrics/indicators/realized_profit_lth_account_based',
    'https://api.glassnode.com/v1/metrics/indicators/realized_loss_lth_account_based',
    'https://api.glassnode.com/v1/metrics/supply/profit_relative'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_supplyinprofit_percentile(df, column='profit_relative', perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate percentile
    percentile = df[column].rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

def calculate_aviv_percentile(df, columns=['investor_capitalization', 'current', 'liveliness', 'price_usd_close'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate AVIV
    aviv = df[columns[2]] * df[columns[1]] * df[columns[3]] / df[columns[0]]
    
    # Calculate percentile
    percentile = aviv.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

def calculate_lthrealized_plratio_logpercentile(df, columns=['realized_profit_lth_account_based', 'realized_loss_lth_account_based'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate LTH realized P/L ratio
    lth_realized_pl = df[columns[0]] / df[columns[1]]
    
    # Apply log transformation
    log_lth_realized_pl = np.log(lth_realized_pl)
    
    # Calculate percentile
    percentile = log_lth_realized_pl.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

# Apply the functions to the merged dataframe with maxmin normalization
merged_df['supplyinprofit_percentile_maxmin'] = calculate_supplyinprofit_percentile(merged_df, normalization='maxmin')
merged_df['aviv_percentile_maxmin'] = calculate_aviv_percentile(merged_df, normalization='maxmin')
merged_df['lthrealized_plratio_logpercentile_maxmin'] = calculate_lthrealized_plratio_logpercentile(merged_df, normalization='maxmin')




In [16]:

# Filter data for the last year
one_year_ago = datetime.now() - timedelta(days=365)
merged_df_last_year = merged_df[merged_df.index > one_year_ago]

# Define a consistent grey color
GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

# Create the visualization
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add price trace
fig.add_trace(
    go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
    secondary_y=False,
)

# Add indicator trace
indicator = merged_df_last_year['supplyinprofit_percentile_maxmin']
fig.add_trace(
    go.Scatter(
        x=merged_df_last_year.index,
        y=indicator,
        name="Supply in Profit Percentile",
        line=dict(color='green', width=2),
        fill='tozeroy',
        fillcolor='rgba(0,255,0,0.1)',
        mode='lines'
    ),
    secondary_y=True,
)

# Add red color for negative values
fig.add_trace(
    go.Scatter(
        x=merged_df_last_year.index,
        y=indicator.where(indicator < 0, 0),
        name="Supply in Profit Percentile (Negative)",
        line=dict(color='red', width=2),
        fill='tozeroy',
        fillcolor='rgba(255,0,0,0.1)',
        mode='lines'
    ),
    secondary_y=True,
)

# Add y=0 line on top (without adding to legend)
fig.add_trace(
    go.Scatter(
        x=merged_df_last_year.index,
        y=[0] * len(merged_df_last_year),
        showlegend=False,
        line=dict(color=GREY_COLOR, width=2),
        hoverinfo='skip'
    ),
    secondary_y=True,
)

# Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
for month in [1, 3, 5, 7, 9, 11]:
    for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
        date = pd.Timestamp(year=year, month=month, day=1)
        if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
            fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

# Get the last value of the indicator
last_value = indicator.iloc[-1]
last_date = indicator.index[-1]

# Determine the color based on the last value
indicator_color = 'green' if last_value >= 0 else 'red'

# Add annotation for the last value
fig.add_annotation(
    x=0.88,  # Place at the right edge of the chart
    y=last_value*1.1,  # Place at the vertical position of the last value
    xref="paper",
    yref="y2",  # Use the secondary y-axis for reference
    text=f"{last_value:.2f}",
    showarrow=False,
    font=dict(size=18, color=indicator_color),
    align="left",
    xanchor="left",
    yanchor="middle",
)

# Update layout
fig.update_layout(
    title={
        'text': "Bitcoin: Supply in Profit Percentile (Last Year)",
        'font': {'color': 'black', 'size': 18, 'weight': 'bold'}  # Changed size to 18 and added bold
    },
    xaxis_title={
        'text': "Date",
        'font': {'color': 'black', 'size': 18}
    },
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0,
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)",
        traceorder="normal",
        itemwidth=40,
        font=dict(size=14)
    ),
    hovermode="x unified",
    plot_bgcolor='white',
    paper_bgcolor='white',
    font={'color': 'black', 'size': 14},
)

# Update traces for legend icons
for trace in fig.data:
    if trace.name != "y=0 Line":
        trace.marker = dict(symbol='circle', size=10)

# Update axes
fig.update_xaxes(
    showgrid=False, 
    tickfont={'color': 'black', 'size': 14},
    zeroline=False
)
fig.update_yaxes(
    showgrid=False, 
    secondary_y=False, 
    tickfont={'color': GREY_COLOR, 'size': 14},
    zeroline=False,
    showline=True,
    linecolor=GREY_COLOR,
    ticks='outside',
    tickcolor=GREY_COLOR,
    title_text='',
    title_font=dict(size=18)
)
fig.update_yaxes(
    showgrid=False, 
    secondary_y=True, 
    range=[-1, 1], 
    tickfont={'color': GREY_COLOR, 'size': 14},
    zeroline=False,
    showline=True,
    linecolor=GREY_COLOR,
    ticks='outside',
    side='right',
    tickcolor=GREY_COLOR,
    title_text='',
    title_font=dict(size=18)
)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
pio.write_html(fig, file='bitcoin_analysis_last_year.html')

# Export the chart as SVG
try:
    svg_bytes = fig.to_image(format="svg")
    with open("bitcoin_analysis_last_year.svg", "wb") as f:
        f.write(svg_bytes)
    print("SVG file 'bitcoin_analysis_last_year.svg' has been created successfully.")
except Exception as e:
    print(f"An error occurred while trying to create the SVG file: {str(e)}")


An error occurred while trying to create the SVG file: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

